In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/traffic data/social status merge/aggreated_collision_with_all_social.csv')
print(df.columns.tolist())


['tract', 'yr_id', '10 to 14', '15 to 17', '18 and 19', '20 to 24', '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', '5 to 9', '50 to 54', '55 to 59', '60 and 61', '62 to 64', '65 to 69', '70 to 74', '75 to 79', '80 to 84', '85 and Older', 'Under 5', '$100,000 to $124,999', '$125,000 to $149,999', '$15,000 to $29,999', '$150,000 to $199,999', '$200,000 or more', '$30,000 to $44,999', '$45,000 to $59,999', '$60,000 to $74,999', '$75,000 to $99,999', 'Less than $15,000', 'population', 'Hispanic', 'Non-Hispanic, Asian', 'Non-Hispanic, Black', 'Non-Hispanic, White', 'Other', 'total_accidents', 'avg_x', 'avg_y']


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Load the dataset
csv_path = "/content/drive/MyDrive/traffic data/social status merge/Ped_Crashes.csv"
df = pd.read_csv(csv_path)

# Convert 'COLLISION_DATE' to datetime
df['COLLISION_DATE'] = pd.to_datetime(df['COLLISION_DATE'], errors='coerce')

# Drop rows with missing location or date
df = df.dropna(subset=['COLLISION_DATE'])

# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Select a specific date to visualize
selected_date = '2020-01-01'
daily_gdf = gdf[gdf['COLLISION_DATE'].dt.date == pd.to_datetime(selected_date).date()]

# Plot the daily crash map
fig, ax = plt.subplots(figsize=(10, 10))
daily_gdf.plot(ax=ax, color='red', markersize=10, alpha=0.7)
plt.title(f"Pedestrian Crashes on {selected_date}")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)
plt.show()



KeyError: ['latitude', 'longitude']

In [ ]:
import pandas as pd
import geopandas as gpd

# Load your collision & social dataset
df = pd.read_csv('/content/drive/MyDrive/traffic data/social status merge/aggreated_collision_with_all_social.csv')

# --- STEP 1: Convert decimal tract to 11-digit GEOID (San Diego County FIPS = 06073) ---
def tract_to_geoid(tract_number):
    try:
        return '06073' + f"{tract_number:.2f}".replace('.', '').zfill(6)
    except:
        return None  # handle non-numeric tract values gracefully

df['GEOID'] = df['tract'].apply(tract_to_geoid)

# --- STEP 2: Load Census Tract Shapefile (CA) and filter to San Diego County ---
tracts = gpd.read_file("/content/drive/MyDrive/traffic data/tl_2022_06_tract.shp")
tracts = tracts[tracts['COUNTYFP'] == '073'].copy()

# Create a proper GEOID in the tract shapefile
tracts['GEOID'] = tracts['STATEFP'] + tracts['COUNTYFP'] + tracts['TRACTCE']

# --- STEP 3: Project to UTM zone 11N (EPSG:26911) for accurate centroids ---
tracts = tracts.to_crs(epsg=26911)
tracts['centroid_x'] = tracts.geometry.centroid.x
tracts['centroid_y'] = tracts.geometry.centroid.y

# --- STEP 4: Merge centroid info into your data using GEOID ---
df_merged = df.merge(
    tracts[['GEOID', 'centroid_x', 'centroid_y']],
    on='GEOID',
    how='left'
)

# --- STEP 5: Fill missing avg_x and avg_y with centroid values ---
df_merged['avg_x'] = df_merged['avg_x'].fillna(df_merged['centroid_x'])
df_merged['avg_y'] = df_merged['avg_y'].fillna(df_merged['centroid_y'])

# Drop temporary columns
df_merged.drop(columns=['centroid_x', 'centroid_y'], inplace=True)

# --- STEP 6: Save to new CSV ---
output_path = '/content/drive/MyDrive/traffic data/social status merge/aggreated_collision_filled_centroids.csv'
df_merged.to_csv(output_path, index=False)

print("✅ Missing coordinates filled using tract centroids and saved to:")
print(output_path)





✅ Missing coordinates filled using tract centroids and saved to:
/content/drive/MyDrive/traffic data/social status merge/aggreated_collision_filled_centroids.csv


In [ ]:
import pandas as pd
import geopandas as gpd

# --- Step 1: Load your CSV ---
csv_path = "/content/drive/MyDrive/traffic data/social status merge/aggreated_collision_with_all_social.csv"
df = pd.read_csv(csv_path)

# --- Step 2: Load the 2020 Census Tract Shapefile ---
shapefile_path = "/content/drive/MyDrive/traffic data/tl_2020_06073_tract20.shp"
tracts = gpd.read_file(shapefile_path)

# --- Step 3: Normalize and calculate centroids ---
tracts = tracts.to_crs(epsg=4326)
tracts['centroid_x'] = tracts.geometry.centroid.x
tracts['centroid_y'] = tracts.geometry.centroid.y

# --- Step 4: Prepare for merging on NAME20 and tract ---
tracts['NAME20'] = pd.to_numeric(tracts['NAME20'], errors='coerce')
df['tract'] = pd.to_numeric(df['tract'], errors='coerce')

# --- Step 5: Merge centroid info using NAME20 = tract ---
df_merged = df.merge(
    tracts[['NAME20', 'centroid_x', 'centroid_y']],
    left_on='tract',
    right_on='NAME20',
    how='left'
)

# --- Step 6: Fill missing avg_x and avg_y with centroid values ---
df_merged['avg_x'] = df_merged['avg_x'].fillna(df_merged['centroid_x'])
df_merged['avg_y'] = df_merged['avg_y'].fillna(df_merged['centroid_y'])

# Clean up temporary columns
df_merged.drop(columns=['centroid_x', 'centroid_y', 'NAME20'], inplace=True)

# --- Step 7: Convert to GeoDataFrame and save as GeoJSON ---
gdf_result = gpd.GeoDataFrame(
    df_merged,
    geometry=gpd.points_from_xy(df_merged['avg_x'], df_merged['avg_y']),
    crs="EPSG:4326"
)

geojson_output = "/content/drive/MyDrive/traffic data/social status merge/new_aggreated_collision_filled_centroids.geojson"
gdf_result.to_file(geojson_output, driver='GeoJSON')

print("✅ GeoJSON with filled centroids saved to:")
print(geojson_output)

# --- Optional: Show a few filled rows ---
gdf_result[['tract', 'avg_x', 'avg_y']].head()




✅ GeoJSON with filled centroids saved to:
/content/drive/MyDrive/traffic data/social status merge/new_aggreated_collision_filled_centroids.geojson


<ipython-input-22-f65bb3f42028>:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tracts['centroid_x'] = tracts.geometry.centroid.x
<ipython-input-22-f65bb3f42028>:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tracts['centroid_y'] = tracts.geometry.centroid.y


,tract,avg_x,avg_y
0,1.00,-117.189822,32.752309
1,2.01,-117.170982,32.750002
2,2.02,-117.176237,32.745479
3,3.01,-117.163032,32.747222
4,3.02,-117.160119,32.746367


In [ ]:
import pandas as pd
import geopandas as gpd

# --- Step 1: Load your CSV ---
csv_path = "/content/drive/MyDrive/traffic data/social status merge/aggreated_collision_with_all_social.csv"
df = pd.read_csv(csv_path)

# --- Step 2: Load the 2020 Census Tract Shapefile ---
shapefile_path = "/content/drive/MyDrive/traffic data/tl_2020_06073_tract20.shp"
tracts = gpd.read_file(shapefile_path)

# --- Step 3: Normalize CRS and calculate tract centroids ---
tracts = tracts.to_crs(epsg=4326)
tracts['tract_centroid_x'] = tracts.geometry.centroid.x
tracts['tract_centroid_y'] = tracts.geometry.centroid.y

# --- Step 4: Prepare for merging ---
tracts['NAME20'] = pd.to_numeric(tracts['NAME20'], errors='coerce')
df['tract'] = pd.to_numeric(df['tract'], errors='coerce')

# --- Step 5: Merge centroid coordinates into df ---
df_merged = df.merge(
    tracts[['NAME20', 'tract_centroid_x', 'tract_centroid_y']],
    left_on='tract',
    right_on='NAME20',
    how='left'
)

# --- Step 6: Save merged data with centroid columns to CSV ---
csv_output = "/content/drive/MyDrive/traffic data/social status merge/new_aggreated_collision_with_tract_centroids.csv"
df_merged.to_csv(csv_output, index=False)

# --- Step 7: Save full tract shapefile with centroid geometry to GeoJSON ---
tracts_centroids = tracts.copy()
tracts_centroids['geometry'] = gpd.points_from_xy(
    tracts['tract_centroid_x'], tracts['tract_centroid_y']
)

geojson_output = "/content/drive/MyDrive/traffic data/social status merge/tract20_centroid_full.geojson"
tracts_centroids.to_file(geojson_output, driver="GeoJSON")

print("✅ CSV with centroid columns saved to:")
print(csv_output)
print("✅ GeoJSON with tract attributes and centroid geometry saved to:")
print(geojson_output)


<ipython-input-2-a203ca31d5da>:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tracts['tract_centroid_x'] = tracts.geometry.centroid.x
<ipython-input-2-a203ca31d5da>:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tracts['tract_centroid_y'] = tracts.geometry.centroid.y


✅ CSV with centroid columns saved to:
/content/drive/MyDrive/traffic data/social status merge/new_aggreated_collision_with_tract_centroids.csv
✅ GeoJSON with tract attributes and centroid geometry saved to:
/content/drive/MyDrive/traffic data/social status merge/tract20_centroid_full.geojson


In [ ]:
import pandas as pd

# Load the CSV
csv_path = "/content/drive/MyDrive/traffic data/social status merge/new_aggreated_collision_with_tract_centroids.csv"
df = pd.read_csv(csv_path)

# Print all column names
print("✅ Columns in the CSV:")
print(df.columns.tolist())


✅ Columns in the CSV:
['tract', 'yr_id', '10 to 14', '15 to 17', '18 and 19', '20 to 24', '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', '5 to 9', '50 to 54', '55 to 59', '60 and 61', '62 to 64', '65 to 69', '70 to 74', '75 to 79', '80 to 84', '85 and Older', 'Under 5', '$100,000 to $124,999', '$125,000 to $149,999', '$15,000 to $29,999', '$150,000 to $199,999', '$200,000 or more', '$30,000 to $44,999', '$45,000 to $59,999', '$60,000 to $74,999', '$75,000 to $99,999', 'Less than $15,000', 'population', 'Hispanic', 'Non-Hispanic, Asian', 'Non-Hispanic, Black', 'Non-Hispanic, White', 'Other', 'total_accidents', 'avg_x', 'avg_y', 'NAME20', 'tract_centroid_x', 'tract_centroid_y']


In [ ]:
import pandas as pd

# Load the CSV
csv_path = "/content/drive/MyDrive/traffic data/social status merge/new_aggreated_collision_with_tract_centroids.csv"
df = pd.read_csv(csv_path)

# Columns to normalize by population
columns_to_normalize = [
    '10 to 14', '15 to 17', '18 and 19', '20 to 24', '25 to 29', '30 to 34', '35 to 39',
    '40 to 44', '45 to 49', '5 to 9', '50 to 54', '55 to 59', '60 and 61', '62 to 64',
    '65 to 69', '70 to 74', '75 to 79', '80 to 84', '85 and Older', 'Under 5',
    '$100,000 to $124,999', '$125,000 to $149,999', '$15,000 to $29,999',
    '$150,000 to $199,999', '$200,000 or more', '$30,000 to $44,999',
    '$45,000 to $59,999', '$60,000 to $74,999', '$75,000 to $99,999',
    'Less than $15,000', 'Hispanic', 'Non-Hispanic, Asian',
    'Non-Hispanic, Black', 'Non-Hispanic, White', 'Other',
    'total_accidents', 'Client Count'  # include this as well
]

# Normalize each by 'population' and create a new column
for col in columns_to_normalize:
    if col in df.columns:
        df[col + '_normalized'] = df[col] / df['population']

# Save the updated DataFrame with new normalized columns
output_path = "/content/drive/MyDrive/traffic data/social status merge/normalized_by_population.csv"
df.to_csv(output_path, index=False)

print("✅ CSV with normalized columns saved to:")
print(output_path)




✅ CSV with normalized columns saved to:
/content/drive/MyDrive/traffic data/social status merge/normalized_by_population.csv


In [3]:
# 📊 This script normalizes columns per 1,000 population, replacing inf values from division by zero with 0.

import pandas as pd
import numpy as np

# Load your CSV
csv_path = "/content/drive/MyDrive/traffic data/social status merge/new_aggreated_collision_with_tract_centroids.csv"
df = pd.read_csv(csv_path)

# Columns to normalize by population
columns_to_normalize = [
    '10 to 14', '15 to 17', '18 and 19', '20 to 24', '25 to 29', '30 to 34', '35 to 39',
    '40 to 44', '45 to 49', '5 to 9', '50 to 54', '55 to 59', '60 and 61', '62 to 64',
    '65 to 69', '70 to 74', '75 to 79', '80 to 84', '85 and Older', 'Under 5',
    '$100,000 to $124,999', '$125,000 to $149,999', '$15,000 to $29,999',
    '$150,000 to $199,999', '$200,000 or more', '$30,000 to $44,999',
    '$45,000 to $59,999', '$60,000 to $74,999', '$75,000 to $99,999',
    'Less than $15,000', 'Hispanic', 'Non-Hispanic, Asian',
    'Non-Hispanic, Black', 'Non-Hispanic, White', 'Other',
    'total_accidents', 'Client Count'
]

# Normalize by population and scale by 1,000 (handle population = 0 safely)
for col in columns_to_normalize:
    if col in df.columns:
        df[col + '_per_1000'] = np.where(
            df['population'] == 0,
            0,
            (df[col] * 1000) / df['population']
        )

# Save to CSV
output_path = "/content/drive/MyDrive/traffic data/social status merge/normalized_by_population_per_1000.csv"
df.to_csv(output_path, index=False)

print("✅ CSV saved with normalized values (inf safely handled):")
print(output_path)



✅ CSV saved with normalized values (inf safely handled):
/content/drive/MyDrive/traffic data/social status merge/normalized_by_population_per_1000.csv


In [ ]:
import pandas as pd
import geopandas as gpd

# --- Step 1: Load normalized CSV ---
csv_path = "/content/drive/MyDrive/traffic data/social status merge/normalized_by_population.csv"
df = pd.read_csv(csv_path)

# --- Step 2: Load shapefile ---
shapefile_path = "/content/drive/MyDrive/traffic data/tl_2020_06073_tract20.shp"
gdf = gpd.read_file(shapefile_path)

# --- Step 3: Convert CRS to EPSG:4326 (standard for GeoJSON) ---
gdf = gdf.to_crs(epsg=4326)

# --- Step 4: Prepare columns for merging ---
df['tract'] = pd.to_numeric(df['tract'], errors='coerce')
gdf['NAME20'] = pd.to_numeric(gdf['NAME20'], errors='coerce')

# --- Step 5: Merge on tract = NAME20 ---
merged_gdf = gdf.merge(df, left_on='NAME20', right_on='tract', how='left')

# --- Step 6: Save as GeoJSON ---
output_geojson = "/content/drive/MyDrive/traffic data/social status merge/tracts_with_normalized_data.geojson"
merged_gdf.to_file(output_geojson, driver="GeoJSON")

print("✅ Merged GeoJSON with normalized CSV data saved to:")
print(output_geojson)


✅ Merged GeoJSON with normalized CSV data saved to:
/content/drive/MyDrive/traffic data/social status merge/tracts_with_normalized_data.geojson


/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: NaN of Infinity value found. Skipped
  ogr_write(


In [ ]:
import geopandas as gpd

geojson_path = "/content/drive/MyDrive/traffic data/social status merge/tracts_with_normalized_data.geojson"
gdf = gpd.read_file(geojson_path)

print(gdf.head())  # Show first 5 rows


  STATEFP20 COUNTYFP20 TRACTCE20      GEOID20  NAME20_x    NAMELSAD20 MTFCC20  \
0        06        073    000100  06073000100      1.00  Census Tract   G5020   
1        06        073    000201  06073000201      2.01  Census Tract   G5020   
2        06        073    000202  06073000202      2.02  Census Tract   G5020   
3        06        073    000301  06073000301      3.01  Census Tract   G5020   
4        06        073    000302  06073000302      3.02  Census Tract   G5020   

  FUNCSTAT20  ALAND20  AWATER20  ... $60,000 to $74,999_normalized  \
0          S  1536251         0  ...                      0.021305   
1          S   864211         0  ...                      0.035029   
2          S  1305997         0  ...                      0.036202   
3          S   401792         0  ...                      0.119644   
4          S   496970         0  ...                      0.051730   

  $75,000 to $99,999_normalized  Less than $15,000_normalized  \
0                      0.04

In [ ]:
# --- STEP A: Count missing BEFORE filling ---
missing_before_x = df['avg_x'].isna().sum()
missing_before_y = df['avg_y'].isna().sum()

# (Already filled in earlier steps...)

# --- STEP B: Count missing AFTER filling ---
missing_after_x = df_merged['avg_x'].isna().sum()
missing_after_y = df_merged['avg_y'].isna().sum()

# --- STEP C: Print Summary ---
print("\n📊 Missing Coordinate Summary:")
print(f"Before filling → avg_x: {missing_before_x}, avg_y: {missing_before_y}")
print(f"After filling  → avg_x: {missing_after_x}, avg_y: {missing_after_y}")



📊 Missing Coordinate Summary:
Before filling → avg_x: 21, avg_y: 21
After filling  → avg_x: 0, avg_y: 0


In [ ]:
# Reload original data (if not already in memory)
df_original = pd.read_csv('/content/drive/MyDrive/traffic data/social status merge/aggreated_collision_with_all_social.csv')

# Identify rows where avg_x or avg_y was originally missing
missing_before = df_original['avg_x'].isna() | df_original['avg_y'].isna()

# Get the filled rows from the updated merged DataFrame
filled_rows = df_merged[missing_before]

# Display selected columns
print("\n🧭 Sample of rows where avg_x or avg_y was filled using tract centroids:")
print(filled_rows[[ 'GEOID', 'avg_x', 'avg_y']].head(20))



🧭 Sample of rows where avg_x or avg_y was filled using tract centroids:
           GEOID       avg_x      avg_y
159  06073008301 -117.232455  32.816188
188  06073008356 -117.139762  32.920915
197  06073008365 -117.144281  32.941120
207  06073008375 -117.200960  32.865868
268  06073009901 -117.245700  32.688676
269  06073009902 -117.208002  32.709652
348  06073013320 -116.936626  32.620834
361  06073013415 -116.998344  32.638375
416  06073015406 -116.898417  32.772304
448  06073016613 -117.004633  32.847392
455  06073016620 -117.000469  32.855900
456  06073016621 -116.998954  32.867672
467  06073016812 -116.835382  32.872286
505  06073017060 -117.049238  33.047490
511  06073017066 -117.124944  33.035417
512  06073017067 -117.071951  32.942807
522  06073017111 -117.173924  33.051776
523  06073017112 -117.186738  33.075305
536  06073017408 -117.273708  33.014875
632  06073019808 -117.262303  33.162228


In [ ]:
# Check if any GEOID in social_df didn't match a tract
unmatched_geoids = social_df[~social_df['GEOID'].isin(tracts_gdf['GEOID20'])]['GEOID'].unique()

print(f"✅ Total GEOID values in CSV: {social_df['GEOID'].nunique()}")
print(f"✅ Total matched GEOIDs in shapefile: {merged_gdf['GEOID'].notna().sum()}")
print(f"❌ Unmatched GEOIDs (not in shapefile): {len(unmatched_geoids)}")

# Optionally show a few examples
if len(unmatched_geoids) > 0:
    print("🔍 Sample unmatched GEOIDs:", unmatched_geoids[:10])


✅ Total GEOID values in CSV: 736
✅ Total matched GEOIDs in shapefile: 0
❌ Unmatched GEOIDs (not in shapefile): 736
🔍 Sample unmatched GEOIDs: ['6073000100' '6073000201' '6073000202' '6073000301' '6073000302'
 '6073000400' '6073000500' '6073000600' '6073000700' '6073000800']


In [ ]:
import geopandas as gpd

shapefile_path = "/content/drive/MyDrive/traffic data/tl_2020_06073_tract20.shp"
gdf = gpd.read_file(shapefile_path)

# Show a few sample rows
gdf.head()


,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,06,073,000100,06073000100,1,Census Tract,G5020,S,1536251,0,+32.7529852,-117.1858795,"POLYGON ((-117.1949 32.75278, -117.19471 32.75..."
1,06,073,000201,06073000201,2.01,Census Tract,G5020,S,864211,0,+32.7540028,-117.1733775,"POLYGON ((-117.17887 32.75765, -117.17797 32.7..."
2,06,073,000202,06073000202,2.02,Census Tract,G5020,S,1305997,0,+32.7453947,-117.1751539,"POLYGON ((-117.18404 32.74571, -117.18383 32.7..."
3,06,073,000301,06073000301,3.01,Census Tract,G5020,S,401792,0,+32.7453524,-117.1653987,"POLYGON ((-117.16864 32.74897, -117.1684 32.74..."
4,06,073,000302,06073000302,3.02,Census Tract,G5020,S,496970,0,+32.7443221,-117.1598012,"POLYGON ((-117.164 32.74091, -117.164 32.74132..."


In [ ]:
import geopandas as gpd
import pandas as pd

# Paths
shapefile_path = "/content/drive/MyDrive/traffic data/tl_2020_06073_tract20.shp"
csv_path = "/content/drive/MyDrive/traffic data/social status merge/aggreated_collision_filled_centroids.csv"

# Load shapefile and CSV
tracts_gdf = gpd.read_file(shapefile_path)
social_df = pd.read_csv(csv_path)

# Convert NAME20 and tract to consistent format (float)
tracts_gdf['NAME20'] = tracts_gdf['NAME20'].astype(str).astype(float)
social_df['tract'] = social_df['tract'].astype(float)

# Merge using NAME20 == tract
merged_gdf = tracts_gdf.merge(social_df, left_on='NAME20', right_on='tract', how='left')

# Save to GeoJSON
output_geojson = "/content/drive/MyDrive/traffic data/merged_by_name20.geojson"
merged_gdf.to_file(output_geojson, driver='GeoJSON')

# Check how many matched
matched_count = merged_gdf['tract'].notna().sum()
total_name20 = tracts_gdf['NAME20'].nunique()

print(f"✅ Merged by NAME20 to tract.")
print(f"📊 Matched tracts: {matched_count} out of {total_name20}")
print(f"📄 Saved GeoJSON to: {output_geojson}")





✅ Merged by NAME20 to tract.
📊 Matched tracts: 736 out of 737
📄 Saved GeoJSON to: /content/drive/MyDrive/traffic data/merged_by_name20.geojson


In [ ]:
import geopandas as gpd

# Load the shapefile
shapefile_path = "/content/drive/MyDrive/traffic data/tl_2020_06073_tract20.shp"
gdf = gpd.read_file(shapefile_path)

# Show column names
print("Available columns in shapefile:")
print(gdf.columns.tolist())


Available columns in shapefile:
['STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'GEOID20', 'NAME20', 'NAMELSAD20', 'MTFCC20', 'FUNCSTAT20', 'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20', 'geometry']


In [ ]:
import pandas as pd
import geopandas as gpd

csv_path = "/content/drive/MyDrive/traffic data/social status merge/agg_collision_with social_filled_centroids.csv"
shapefile_path = "/content/drive/MyDrive/traffic data/tl_2022_06_tract.shx"

df = pd.read_csv(csv_path)
gdf = gpd.read_file(shapefile_path)

def geo_to_short(tractce):
    tractce = tractce.strip()
    if len(tractce) > 4:
        return float(f"{tractce[:2]}.{tractce[2:]}")
    else:
        return float(tractce)

gdf["tract"] = gdf["TRACTCE"].apply(geo_to_short)
merged = gdf.merge(df, on="tract", how="left")

# Save to GeoJSON
merged.to_file("/content/drive/MyDrive/traffic data/sd_social_status_faces.geojson", driver="GeoJSON")
print("✅ GeoJSON saved to: sd_social_status_faces.geojson")




✅ GeoJSON saved to: sd_social_status_faces.geojson


In [ ]:
import geopandas as gpd
import pandas as pd

# Paths
csv_path = "/content/drive/MyDrive/traffic data/social status merge/agg_collision_with social_filled_centroids.csv"
shapefile_path = "/content/drive/MyDrive/traffic data/tl_2022_06_tract.shx"  # Make sure all .shp, .shx, .dbf are in the same folder

# Load data
collision_df = pd.read_csv(csv_path)
tracts_gdf = gpd.read_file(shapefile_path)

# Filter for San Diego County (County FIPS = 073 in California = state FIPS 06)
tracts_gdf = tracts_gdf[(tracts_gdf["STATEFP"] == "06") & (tracts_gdf["COUNTYFP"] == "073")]
# Ensure both GEOID columns are strings
collision_df["GEOID"] = collision_df["GEOID"].astype(str)
tracts_gdf["GEOID"] = tracts_gdf["GEOID"].astype(str)

# Merge
merged_gdf = tracts_gdf.merge(collision_df, on="GEOID", how="inner")

# Merge with collision data on GEOID
merged_gdf = tracts_gdf.merge(collision_df, on="GEOID", how="inner")

# Output path
output_geojson = "/content/drive/MyDrive/traffic data/agg_collision_san_diego.geojson"

# Save as GeoJSON
merged_gdf.to_file(output_geojson, driver="GeoJSON")

print("GeoJSON file saved:", output_geojson)



GeoJSON file saved: /content/drive/MyDrive/traffic data/agg_collision_san_diego.geojson


/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/lib/python3.11/dist-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [ ]:
print(tracts_gdf.columns)


Index(['geometry'], dtype='object')


In [ ]:

merged_gdf.to_file(shapefile_path)

print("✅ Shapefile created at:", output_path)

<ipython-input-21-15ad0af5727d>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_gdf.to_file(shapefile_path)
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '85 and Older' to '85 and Old'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '$100,000 to $124,999' to '$100,000 t'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '$125,000 to $149,999' to '$125,000 t'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '$15,000 to $29,999' to '$15,000 to'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '$150,000 to $199,999' to '$150,000 t'
  ogr_write(
/usr/local/lib/python3.11/dist-pa

✅ Shapefile created at: /mnt/data/sd_social_status_tracts.shp
